In [1]:
# !pip install transformers --upgrade
from transformers import pipeline, AutoModelForSeq2SeqLM, AutoTokenizer
import os
import pandas as pd
import datasets
import gc
from tqdm import tqdm

In [2]:
# model_ckpt = 'Helsinki-NLP/opus-mt-ko-en'
model_ckpt2= 'circulus/kobart-trans-ko-en-v2' #bart`
# model_ckpt3 = 'alphahg/opus-mt-ko-en-finetuned-ko-to-en100'
# pipe2 = pipeline('translation', model=model_ckpt2, device=0)

In [6]:
pipe = pipeline('translation', model = model_ckpt2, device=0,max_length=256)

In [7]:
# test
pipe('인간 바둑기사가 인공지능에 승리할 수 있었던 이유는 인공지능으로부터 전술을 직접 물려 받았기 때문이다.')

[{'translation_text': 'The reason why human Go players were able to win artificial intelligence was because they inherited tactics directly from artificial intelligence'}]

---
# kor -> eng backtranslation 작업 순서

### 작업 할당 :

- `김시혁` :  'jd' 전라도
- `정선아` :  'kw' 강원도
- `임종인` :  'gs' 경상도
- `이승용` :  'jj' 제주도
- `신덕식` :  'cc' 충청도

---

### `data_save(dataset, pipe, '지역', 작업크기, 시작index)`
>1. ### `dataset` : 데이터 셋 [dataframe : pandas dataframe]
>2. ### `pipe` : hugging face pipeline [func : huggingface pipeline function]
>3. ### `지역` : 사투리 지역 [str : 'jj', 'gs', 'kw', 'cc', 'jd']
>4. ### `작업크기` : num of rows [int]
>5. ### `directory` : 저장 경로 [str : directory]
>>예) data_save(df,pipe,'jj',5000,0,'/aiffel/aiffelthon/') -> '지역', 시작index만 변경해주시면 됩니다.

In [64]:
directory = os.getenv('HOME') + '/aiffel/aiffelthon/' # 데이터 경로 지정

In [127]:
df = pd.read_csv(directory+'datset_full_raw_needs_translation_0221_v2.csv') # 데이터 파일 불러오기
df.drop_duplicates(inplace=True)
del df['Unnamed: 0']
df['text'] = df['text'].str.replace('[\(\)]','',regex=True)
df['dial'] = df['dial'].str.replace('[\(\)]','',regex=True)
reg=['jj','jd','gs','kw','cc']
df = df.sample(frac=1)
dfs = pd.DataFrame(columns=['text', 'dial', 'reg', 'pair'])
for i in reg :
    temp = df.loc[df['reg']==i]
    dfs = dfs.merge(temp[:200000],how='outer')
dfs = dfs.sample(frac=1,random_state=1)
dfs.dropna(inplace=True)
dfs.head()

,text,dial,reg,pair
276826,영화에서 보면 뒷발 차기 해 가지고 이게,영화에서 보면 뒷발 차기 해 가지고잉 이게,jd,(가지고잉)/(가지고)
849425,이게 나이별 나이에 따라서 이게 옷 입는 것도 바뀌는 거 같애.,이게 나이별 나이에 따라서 이게 옷 입는 것도 바껴지는 거 같애.,cc,(바껴지는)/(바뀌는)
504499,일단 뭐하는 연애혁명이나,일단 머하는 연애혁명이나,gs,(머하는)/(뭐하는)
601054,뭔가 잘못됐어.,뭔가 잔못됐어.,kw,(잔못됐어.)/(잘못됐어.)
980221,그래서 그거를 그냥 썩혀 두기에는 너무 그러니까 당근 마켓이나,그래서 그거를 그냥 썩혀 두기에는 너무 그르니까 당근 마켓이나,cc,(그르니까)/(그러니까)


In [151]:
def data_save(data, pipe, prefix, num_rows,start, directory) :
    
    '''
    
    1. `dataset` : 데이터 셋 [dataframe : pandas dataframe]
    2. `pipe` : hugging face pipeline [func : huggingface pipeline function]
    3. `지역` : 사투리 지역 [str : 'jj', 'gs', 'kw', 'cc', 'jd']
    4. `작업크기` : num of rows [int]
    5. `directory` : 저장 경로 [str : directory]
    
        예) data_save(df,pipe,'jj',5000,0,'/aiffel/aiffelthon/') -> '지역', 시작index만 변경해주시면 됩니다.
        
    '''
    
    df_final = pd.DataFrame(columns=['text', 'dial', 'reg', 'pair','eng'])

    for i in range(start,len(data), num_rows) :
        dataset = data[i : i +num_rows].reset_index()
        dataset = datasets.Dataset.from_pandas(dataset)
        dataset = dataset.map(lambda ds : {'eng' : pipe(ds['text'])[0]['translation_text']})
        dataset.set_format('pandas')
        df_final = df_final.merge(dataset[:], how='outer')
        df_final.to_csv(directory + f'dataset_en_ko_{prefix}_{i}_{i+num_rows}.csv', index=False)
        print(f'작업 완료 : {prefix}_{i}_{i+num_rows}' )
        print(f'저장 경로 : {directory}dataset_ko_en_{prefix}_{i}_{i+num_rows}.csv')

### 본인이 해당하는 부분만 주석 풀고 데이터 만들어주세요~

In [132]:
# dfs1 = dfs[:200000].copy() # 김시혁
# dfs1 = dfs[200000:400000].copy() # 정선아
# dfs1 = dfs[400000:600000].copy() # 임종인
# dfs1 = dfs[600000:800000].copy() # 이승용
# dfs1 = dfs[800000:].copy() # 이승용

cc    40279
jj    40031
gs    39926
jd    39912
kw    39851
Name: reg, dtype: int64

In [ ]:
name = 'sy'# <<<<<<--------이름 꼭 바꿔주세요------------------------------------<<<<<<<<<<<<<<
date = '0221' # <<<<<<--------작업 일자로 바꿔주세요------------------------------------<<<<<<<<<<<<<<

prefix = f'translated_{name}' # 지역
num_rows = 5000
data_save(dfs1,pipe,prefix,num_rows,0,directory)

  0%|          | 0/5000 [00:00<?, ?ex/s]

작업 완료 : translated_45000_50000
저장 경로 : /aiffel/aiffel/aiffelthon/dataset_ko_en_translated_45000_50000.csv


  0%|          | 0/5000 [00:00<?, ?ex/s]

작업 완료 : translated_50000_55000
저장 경로 : /aiffel/aiffel/aiffelthon/dataset_ko_en_translated_50000_55000.csv


  0%|          | 0/5000 [00:00<?, ?ex/s]

...

  0%|          | 0/5000 [00:00<?, ?ex/s]

작업 완료 : translated_145000_150000
저장 경로 : /aiffel/aiffel/aiffelthon/dataset_ko_en_translated_145000_150000.csv


  0%|          | 0/5000 [00:00<?, ?ex/s]

작업 완료 : translated_150000_155000
저장 경로 : /aiffel/aiffel/aiffelthon/dataset_ko_en_translated_150000_155000.csv


  0%|          | 0/5000 [00:00<?, ?ex/s]

In [91]:
import glob
files = glob.glob(directory+f'*jd*.csv')
files

['/aiffel/aiffel/aiffelthon/dataset_en_ko_jd_30000_40000.csv',
 '/aiffel/aiffel/aiffelthon/dataset_en_ko_jd_80000_90000.csv',
 '/aiffel/aiffel/aiffelthon/dataset_en_ko_jd_20000_30000.csv',
 '/aiffel/aiffel/aiffelthon/dataset_en_ko_jd_0_10000.csv',
 '/aiffel/aiffel/aiffelthon/dataset_en_ko_jd_90000_100000.csv',
 '/aiffel/aiffel/aiffelthon/dataset_en_ko_jd_10000_20000.csv',
 '/aiffel/aiffel/aiffelthon/dataset_en_ko_jd_70000_80000.csv',
 '/aiffel/aiffel/aiffelthon/dataset_en_ko_jd_50000_60000.csv',
 '/aiffel/aiffel/aiffelthon/dataset_en_ko_jd_40000_50000.csv']

In [94]:
# data merge하기
df_final = pd.DataFrame(columns=['text', 'dial', 'reg', 'pair','eng'])
for i in files :
    df_new = pd.read_csv(i, usecols =['text', 'dial', 'reg', 'pair','eng'])
    df_final = df_final.merge(df_new, how='outer')
df_final.head()
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100002 entries, 0 to 100001
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    100002 non-null  object
 1   dial    100002 non-null  object
 2   reg     100002 non-null  object
 3   pair    100002 non-null  object
 4   eng     100002 non-null  object
dtypes: object(5)
memory usage: 4.6+ MB


# 통합 파일 'added' 경로의 저장하기

In [61]:
import shutil

In [95]:
df_final.to_csv(directory + f'dataset_full_ko_en_added_{date}_{name}.csv' , index=False)

src= directory
dest = directory + 'added/'

os.makedirs(dest,exist_ok=True)

new_file = [i[26:] for i in glob.glob(directory+'*added*.csv')]
for i in new_file :
    shutil.move(src + i, dest + i)

In [96]:
# 통합 전 파일 별도 저장

old_files = [i[26:] for i in glob.glob(directory+'*translated*.csv')]

dest = diretory + 'past_files/'

os.makedirs(dest,exist_ok=True)

for i in old_files :
    shutil.move(src + i, dest + i)